In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

In [3]:
# I created a function where you enter yahoo ticker name
# and it calculates volatilty, volatility moving averages
# aligns these calculations to dates when hurricanes have hit
# the US and then returns them in a pandas dataframe

In [4]:
def align_my_data(yahoo_ticker):
    # this first part of the function calculates my measure of volatility
    # from the Yfinance ticker entered
    name = yahoo_ticker
    x = yf.Ticker(yahoo_ticker).history(period='MAX')
    high = x['High']
    low = x['Low']
    mean = (abs(high)+abs(low))/2
    dr = abs(high-low)
    ans = ((dr/mean)*1000)+10
    hi = pd.DataFrame(data=x)
    hi['Wills Volatility Measure'] = ans
    hi['Date'] = hi.index
    
    # I needed to export this to a CSV and read back in for it to work
    hi.to_csv('in progress.csv') 
    data = pd.read_csv('in progress.csv')
    
    # this next section of the function aligns stock volatility data to hurricane data
    # it takes inputs of the dataframe created in the first section of the function
    # and the ticker name entered
    
    def Align_my_data(data, name):
        hurricanes = pd.read_csv('hurricanes_scraped.csv') # reads in hurricane data
        hurricanes = hurricanes.rename(columns={'Date of closest approach': 'Date'})
        for x in range(0,len(hurricanes)): # this loop formats hurricane data to match format of volatility data
            hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
        zing = data
        
        # I will now create 21 point (day) moving average series
        MA21 = [] 
        for y in range(0,21):
            MA21.append('NaN')
        for x in range(21,len(zing)):
            MA21.append((sum(zing['Wills Volatility Measure'][x-21:x])/21))
        zing['MA21'] = MA21 # this is a list of the moving average calculations
        
        # I will also calculate a 21-point (21 trading days in a month) trailing moving average series
        Trailing21 = [] 
        for y in range(0,35):
            Trailing21.append('NaN')
        for x in range(35,len(zing)):
            Trailing21.append((sum(zing['Wills Volatility Measure'][x-35:x-14])/21))
        zing['Trailing MA'] = Trailing21 # this is a list of the trailing MA
        
        # I will create empty lists to append same number of data points to each
        # from my loops, so all lists will be the same length and aligned correctly
        zing_box = []
        ticker = []
        date_box = []
        hurricane_box = []
        MA21_box = []
        Trailing21_box = []
        
        diff_ma_box = []
        diff_tma_box = []

        # this next part of the function uses boolean to check whether
        # dates match, if they do then it appends data required to our empty lists
        for x in range(0,len(hurricanes['Date'])): # loops over hurricane hit dates
            for y in range(0, len(zing['Date'])): # loops over dates of time series to align with hurricanes
                if hurricanes['Date'][x] == zing['Date'][y]: # uses boolean to check whether dates match
                    zing_box.append(zing['Wills Volatility Measure'][y])
                    date_box.append(hurricanes['Date'][x])
                    hurricane_box.append(hurricanes['Saffir-Simpson Category'][x])
                    MA21_box.append(zing['MA21'][y])
                    Trailing21_box.append(zing['Trailing MA'][y])
                    # next I will calculate the differnce between volatility measure and their moving avareges
                    diff_ma_box.append((zing['Wills Volatility Measure'][y]-zing['MA21'][y]))
                    diff_tma_box.append((zing['Wills Volatility Measure'][y]-zing['Trailing MA'][y]))
                    ticker.append(name)
                    
        # I will combine all the lists created into a Pandas dataframe
        result = pd.DataFrame({'Symbol': ticker, 'Hurricane Date': date_box,'Hurricane Category': hurricane_box,'Wills Volatility Measure': zing_box, 'Volatility Measure MA': MA21_box, 'Volatility Measure Trailing MA': Trailing21_box, 'MA Difference': diff_ma_box, 'TMA Difference': diff_tma_box})
        
        # Cleans data for zero values
        # calculations are to a very high degree of accuracy
        # therefore any zero values in moving average differences will be removed
        for x in range(0,len(result)): # now we need to drop zero values
            if result['MA Difference'][x] == 0:
                result.drop(index = [x] , inplace = True)
        
        # return dataframe we created to the outer function
        return(result)
    
    # return results
    return(Align_my_data(data,name))

In [5]:
sandp = align_my_data("^GSPC")

<ipython-input-4-fb6d17e5f698>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [12]:
sandp.head(6) # great, this fuction works!

,Symbol,Hurricane Date,Hurricane Category,Wills Volatility Measure,Volatility Measure MA,Volatility Measure Trailing MA,MA Difference,TMA Difference
0,^GSPC,2003-09-18,2,23.950302,20.233923,21.249859,3.716379,2.700443
1,^GSPC,2004-09-16,3,15.065872,17.962117,20.382792,-2.896245,-5.316920
2,^GSPC,2004-08-13,4,16.446446,21.157197,19.375114,-4.710751,-2.928668
3,^GSPC,2004-09-16,3,15.065872,17.962117,20.382792,-2.896245,-5.316920
4,^GSPC,2005-08-29,3,20.555466,17.848863,16.947224,2.706603,3.608242
5,^GSPC,2005-08-25,3,13.433363,18.010988,17.388075,-4.577625,-3.954712


In [ ]:
# The run time is not great for this function
# therefore, I will not use a loop to put through indices all at once.
# I will now extract and align indices data for analysis using this function
# I will then concatonate them into a one dataframe

In [10]:
dji = align_my_data("^DJI")

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [11]:
nasdaq = align_my_data("^IXIC")

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [12]:
rus2000 = align_my_data("^RUT")

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [13]:
ftse = align_my_data("^FTSE")

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [14]:
dax = align_my_data("^GDAXI")

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [15]:
cac = align_my_data("^FCHI")

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


In [40]:
stoxx50 = align_my_data("^STOXX50E")
len(stoxx50)

<ipython-input-3-062657c24c53>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")


3

In [ ]:
# I will concat these dataframe into one file

In [41]:
indicies = pd.concat([sandp,dji,nasdaq,rus2000,ftse,dax,cac], axis = 0)

In [42]:
indicies

,Symbol,Hurricane Date,Hurricane Category,Wills Volatility Measure,Volatility Measure MA,Volatility Measure Trailing MA,MA Difference,TMA Difference
0,^GSPC,2003-09-18,2,23.950302,20.233923,21.249859,3.716379,2.700443
1,^GSPC,2004-09-16,3,15.065872,17.962117,20.382792,-2.896245,-5.316920
2,^GSPC,2004-08-13,4,16.446446,21.157197,19.375114,-4.710751,-2.928668
3,^GSPC,2004-09-16,3,15.065872,17.962117,20.382792,-2.896245,-5.316920
4,^GSPC,2005-08-29,3,20.555466,17.848863,16.947224,2.706603,3.608242
...,...,...,...,...,...,...,...,...
26,^FCHI,1998-08-27,2,38.584618,30.090427,24.317934,8.494191,14.266684
27,^FCHI,1999-09-16,2,20.941558,23.530387,25.593046,-2.588830,-4.651488
28,^FCHI,1999-10-18,2,23.120406,23.337838,22.954276,-0.217432,0.166130
29,^FCHI,1999-08-23,3,22.021855,25.527241,25.139610,-3.505386,-3.117755


In [ ]:
# I will rename ticker symbols to the index name

In [43]:
indicies['Symbol'] = indicies['Symbol'].replace('^GSPC', 'S&P 500')

In [44]:
indicies['Symbol'] = indicies['Symbol'].replace('^DJI', 'Dow Jones 30')
indicies['Symbol'] = indicies['Symbol'].replace('^IXIC', 'Nasdaq')
indicies['Symbol'] = indicies['Symbol'].replace('^RUT', 'Russel 2000')
indicies['Symbol'] = indicies['Symbol'].replace('^FTSE', 'FTSE 100')
indicies['Symbol'] = indicies['Symbol'].replace('^GDAXI', 'DAX')
indicies['Symbol'] = indicies['Symbol'].replace('^FCHI', 'CAC 40')

In [ ]:
# Great, this dataset is cleaned and aligned
# I will now export it as a csv file

In [46]:
indicies.to_csv('indicies.csv')